In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras as K
from keras.layers import *
import matplotlib.pyplot as plt
import numpy as np


tf.keras.backend.set_floatx("float64")

In [3]:
def import_data():

    ds_train,info = tfds.load('caltech_birds2010', split='train',with_info=True)
    ds_test,info = tfds.load('caltech_birds2010', split='test',with_info=True)
    
    return ds_train,ds_test,info

In [4]:
ds_train_raw,ds_test_raw,info = import_data()

In [5]:
IMAGE_SIZE = 128
def pre_process_image(record):
    
    image = record["image"]
    image = image / 255
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = tf.cast(image, tf.float32)
    
    seg_image = record["segmentation_mask"]
    seg_image = tf.image.resize(seg_image, (IMAGE_SIZE, IMAGE_SIZE))
    seg_image = tf.cast(seg_image, tf.float32)
    return image,seg_image

resize_train = ds_train_raw.map(pre_process_image)

resize_test = ds_test_raw.map(pre_process_image)

In [6]:
input_img = Input(next(iter(resize_train))[0].shape, name="input_img") 

In [7]:

def dice_coef(y_true, y_pred):
    y_true_f = K.backend.flatten(y_true)
    y_pred_f = K.backend.flatten(y_pred)
    intersection = K.backend.sum(y_true_f * y_pred_f)

    return (2. * intersection) / (K.backend.sum(y_true_f) + K.backend.sum(y_pred_f))

def diceCoefLoss(y_true, y_pred):
    return (1-dice_coef(y_true, y_pred))

In [30]:
x = Conv2D(8, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
decoded = Conv2D(1, (2, 2), activation='sigmoid', padding='same',name="decoded")(x)
    
autoencoder = K.Model(input_img, decoded)
optimizer = K.optimizers.adam_v2.Adam(learning_rate=1e-6)
autoencoder.compile(optimizer=optimizer, loss=diceCoefLoss,metrics=[K.metrics.MeanIoU(num_classes=2),dice_coef])

In [31]:
autoencoder.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_img (InputLayer)      [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_28 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 32, 32, 32)        4640

In [36]:
callbacks = [
    K.callbacks.ModelCheckpoint("../bird_autoencoder.h5", save_best_only=True, monitor="val_dice_coef",mode="max")
]

# Train the model, doing validation at the end of each epoch.
epochs = 20


In [37]:
ds_train = resize_train.batch(64).prefetch(tf.data.AUTOTUNE)
ds_test = resize_test.batch(64).prefetch(tf.data.AUTOTUNE)

In [38]:
ds_train

<PrefetchDataset shapes: ((None, 128, 128, 3), (None, 128, 128, 1)), types: (tf.float32, tf.float32)>

In [ ]:
autoencoder.fit(ds_train,validation_data=ds_test,epochs=epochs,verbose=2,callbacks=callbacks)

Epoch 1/20
47/47 - 273s - loss: 0.7407 - mean_io_u_3: 0.4168 - dice_coef: 0.2593 - val_loss: 0.7523 - val_mean_io_u_3: 0.4217 - val_dice_coef: 0.2472 - 273s/epoch - 6s/step
Epoch 2/20
47/47 - 314s - loss: 0.7401 - mean_io_u_3: 0.4168 - dice_coef: 0.2599 - val_loss: 0.7517 - val_mean_io_u_3: 0.4217 - val_dice_coef: 0.2478 - 314s/epoch - 7s/step
Epoch 3/20
47/47 - 325s - loss: 0.7395 - mean_io_u_3: 0.4168 - dice_coef: 0.2605 - val_loss: 0.7512 - val_mean_io_u_3: 0.4217 - val_dice_coef: 0.2484 - 325s/epoch - 7s/step
Epoch 4/20
47/47 - 273s - loss: 0.7389 - mean_io_u_3: 0.4168 - dice_coef: 0.2611 - val_loss: 0.7506 - val_mean_io_u_3: 0.4217 - val_dice_coef: 0.2490 - 273s/epoch - 6s/step
Epoch 5/20
47/47 - 296s - loss: 0.7383 - mean_io_u_3: 0.4168 - dice_coef: 0.2618 - val_loss: 0.7500 - val_mean_io_u_3: 0.4217 - val_dice_coef: 0.2495 - 296s/epoch - 6s/step
Epoch 6/20
47/47 - 320s - loss: 0.7376 - mean_io_u_3: 0.4168 - dice_coef: 0.2624 - val_loss: 0.7494 - val_mean_io_u_3: 0.4217 - val_dic

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["loss"],label= "train_loss")
plt.plot(history.history["val_loss"],label="val_loss")
plt.legend()